### Load packages

In [19]:
%load_ext autoreload
%autoreload 2

import duckdb as ddb
from pupil_classes import (
    Athlete,
    Blinks,
    Fixations,
    FixationsOnSurfaces,
    GazePositions,
    GazePositionsOnSurfaces,
    HeadPosesTracker,
    PupilsPositions,
    Test,
    AimTracker
)
from datetime import date, datetime


DB_LOCATION = "../../data/pupils/db5.duckdb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Database session

In [20]:
con = ddb.connect(database=DB_LOCATION, read_only=False)

In [21]:
from dataclasses import fields

def generate_table_creation_sql() -> str:
    table_class_name_map = {
        "Athlete": "athlets",
        "Test": "tests",
        "Blinks": "blinks",
        "GazePositions": "gaze_positions",
        "PupilsPositions": "pupils_positions",
        "Fixations": "fixations",
        "FixationsOnSurfaces": "fixations_on_surfaces",
        "GazePositionsOnSurfaces": "gaze_positions_on_surfaces",
        "HeadPosesTracker": "head_poses_tracker",
        "AimTracker": "aim_tracker"
    }

    data_classes = [
        Athlete,
        Test,
        Blinks,
        GazePositions,
        PupilsPositions,
        Fixations,
        FixationsOnSurfaces,
        GazePositionsOnSurfaces,
        HeadPosesTracker,
        AimTracker
    ]

    sql_script = ""

    for data_class in data_classes:
        table_name = table_class_name_map[data_class.__name__]
        # Get the name of the dataclass
        class_name = data_class.__name__

        # Get the fields of the dataclass
        data_class_fields = fields(data_class)

        # Initialize SQL statement
        sql_statement = f"CREATE TABLE {table_name} ("

        # Loop through fields
        for field in data_class_fields:
            field_name = field.name
            field_type = field.type

            # Determine SQL type based on Python type
            sql_type = "VARCHAR"
            if field_type == str:
                sql_type = "VARCHAR"
            elif field_type == int:
                sql_type = "INTEGER"
            elif field_type == float:
                sql_type = "FLOAT"
            elif field_type == date:
                sql_type = "DATE"
            elif field_type == datetime:
                sql_type = "TIMESTAMP"

            # Add field to SQL statement
            sql_statement += f"{field_name} {sql_type}, "

        # # Define primary key
        # logger.info(table_name)
        # if table_name == "pupil_positions":
        #     sql_statement = f"{sql_statement[:-2]})"

        # elif table_name == "head_pose_tracker":
        #     sql_statement += "PRIMARY KEY (test_id, timestamp))"

        # elif table_name == "gaze_positions":
        #     sql_statement += "PRIMARY KEY (test_id, gaze_timestamp))"

        # elif "fixations_on_surface" in table_name:
        #     sql_statement = f"{sql_statement[:-2]})"

        # elif "gaze_positions_on_surface" in table_name:
        #     sql_statement = f"{sql_statement[:-2]})"

        # elif (table_name == "fixations") | (table_name == "blinks"):
        #     sql_statement += "PRIMARY KEY (id, test_id, start_timestamp))"

        # elif table_name == "users":
        #     sql_statement += "PRIMARY KEY (id))"

        # elif table_name == "tests":
        #     sql_statement += "PRIMARY KEY (id, user_id))"

        # sql_script += f"{sql_statement};\n"

        # Define primary key

        logger.info(table_name)
        if table_name == "pupils_positions":
            sql_statement += "PRIMARY KEY (test_id, method, eye_id, pupil_timestamp))"
            # sql_statement = f"{sql_statement[:-2]})"

        elif table_name == "head_poses_tracker":
            sql_statement += "PRIMARY KEY (test_id, timestamp))"
            # sql_statement = f"{sql_statement[:-2]})"

        elif table_name == "gaze_positions":
            sql_statement += "PRIMARY KEY (test_id, gaze_timestamp, base_data))"
            # sql_statement = f"{sql_statement[:-2]})"

        elif "fixations_on_surfaces" in table_name:
            sql_statement += (
                "PRIMARY KEY (test_id, world_index, start_timestamp, surface))"
            )
            # sql_statement = f"{sql_statement[:-2]})"

        elif "gaze_positions_on_surfaces" in table_name:
            sql_statement += (
                "PRIMARY KEY (test_id, surface, world_index, gaze_timestamp, confidence))"
            )
            # sql_statement = f"{sql_statement[:-2]})"

        elif (table_name == "fixations") | (table_name == "blinks"):
            sql_statement += "PRIMARY KEY (test_id, id, start_timestamp))"
            # sql_statement = f"{sql_statement[:-2]})"

        elif table_name == "athlets":
            sql_statement += "PRIMARY KEY (id))"
            # sql_statement = f"{sql_statement[:-2]})"

        elif table_name == "tests":
            sql_statement += "PRIMARY KEY (id, athlete_id))"
            # sql_statement = f"{sql_statement[:-2]})"

        elif table_name == "aim_tracker":
            sql_statement += "PRIMARY KEY (test_id))"
            # sql_statement = f"{sql_statement[:-2]})"



        sql_script += f"{sql_statement};\n"

    return sql_script


query = generate_table_creation_sql()

athlets
tests
blinks
gaze_positions
pupils_positions
fixations
fixations_on_surfaces
gaze_positions_on_surfaces
head_poses_tracker
aim_tracker


### Build Database

In [22]:
logger.info(query)

CREATE TABLE athlets (id VARCHAR, name VARCHAR, gender VARCHAR, birth_date VARCHAR, side VARCHAR, height FLOAT, weight FLOAT, discipline VARCHAR, discipline_category VARCHAR, main_position VARCHAR, second_position VARCHAR, club VARCHAR, PRIMARY KEY (id));
CREATE TABLE tests (id VARCHAR, athlete_id VARCHAR, category VARCHAR, test_name VARCHAR, variation VARCHAR, test_date TIMESTAMP, test_time TIMESTAMP, upload_datetime TIMESTAMP, duration TIMESTAMP, PRIMARY KEY (id, athlete_id));
CREATE TABLE blinks (test_id VARCHAR, id VARCHAR, start_timestamp TIMESTAMP, duration FLOAT, end_timestamp TIMESTAMP, start_frame_index INTEGER, index INTEGER, end_frame_index INTEGER, confidence FLOAT, filter_response VARCHAR, base_data VARCHAR, PRIMARY KEY (test_id, id, start_timestamp));
CREATE TABLE gaze_positions (test_id VARCHAR, gaze_timestamp TIMESTAMP, world_index INTEGER, confidence FLOAT, norm_pos_x FLOAT, norm_pos_y FLOAT, base_data VARCHAR, gaze_point_3d_x FLOAT, gaze_point_3d_y FLOAT, gaze_point_3

In [23]:
con.sql(query)
con.close()